In [1]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

c:\hyunkwon\robot_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## GPU 사용을 위해 확인하는 용도의 코드들

print(f"PyTorch 버전: {torch.__version__}") # 버전 이름 뒤에 +cu118이나 +cu121 같은 글자가 없고 숫자만 있다면 CPU 전용 버전이 설치된 것
print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")  # True가 나와야 GPU 전용 torch로 진행 가능 # False라면 커널 재설정 후 다시 시도

# GPU용 pytorch 설치 후 다시 커널 실행해서 확인할 부분
print(f"사용 중인 GPU 이름: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

PyTorch 버전: 2.9.1+cpu
CUDA 사용 가능 여부: False


In [2]:
# 0. 장치 설정 (GPU 최우선, 없으면 CPU)
# --- [설정부] ---

IMG_DIR = './img'
MASK_DIR = './mask'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 4
EPOCHS = 50
LR = 0.0001

print(f"현재 사용 중인 장치: {DEVICE}")

현재 사용 중인 장치: cpu


In [3]:
# --- [1. 데이터셋 정의] ---
class ToolWearDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        # 이미지 파일 리스트 가져오기
        self.images = [f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name) # 파일명 동일 가정

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 마스크가 없을 경우를 대비한 예외처리
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = mask / 255.0
        else:
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            
        return image, mask.float().unsqueeze(0), img_name

In [4]:
# --- [2. 변환 및 모델 준비] ---
transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.Normalize(),
    ToTensorV2(),
])

model = smp.Unet(
    encoder_name="mobilenet_v2",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation='sigmoid'
).to(DEVICE)

criterion = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scaler = GradScaler()

'[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1017)' thrown while requesting HEAD https://huggingface.co/smp-hub/mobilenet_v2.imagenet/resolve/e67aa804e17f7b404b629127eabbd224c4e0690b/config.json
Retrying in 1s [Retry 1/5].
c:\hyunkwon\robot_test\lib\site-packages\segmentation_models_pytorch\encoders\__init__.py:136: UserWarning: Error loading mobilenet_v2 `imagenet` weights from Hugging Face Hub, trying loading from original url...
  warnings.warn(message, UserWarning)
C:\Users\KWON HYUN\AppData\Local\Temp\ipykernel_34960\1350571419.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
c:\hyunkwon\robot_test\lib\site-packages\torch\cuda\amp\grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


In [5]:
# --- [3. 학습 함수] ---
def train():
    dataset = ToolWearDataset(IMG_DIR, MASK_DIR, transform=transform)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    print(f"학습 시작 장치: {DEVICE}")
    model.train()
    for epoch in range(EPOCHS):
        for imgs, masks, _ in loader:
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            optimizer.zero_grad()
            with autocast(enabled=(DEVICE.type == 'cuda')):
                outputs = model(imgs)
                loss = criterion(outputs, masks)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {loss.item():.4f}")
    
    torch.save(model.state_dict(), "tool_model.pth")
    print("모델 저장 완료.")

In [6]:
# --- [4. 결과 확인(추론) 함수] ---
def predict_and_show():
    model.eval()
    test_img_name = os.listdir(IMG_DIR)[0] # 첫 번째 이미지로 테스트
    image = cv2.imread(os.path.join(IMG_DIR, test_img_name))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 전처리
    h, w = image.shape[:2]
    input_img = transform(image=image_rgb)['image'].unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        output = model(input_img)
        # 다시 원래 크기로 복구
        pred_mask = torch.nn.functional.interpolate(output, size=(h, w), mode='bilinear')
        pred_mask = (pred_mask > 0.5).cpu().numpy().squeeze()
    
    # 시각화: 원본 위에 마스크를 붉은색으로 덧씌움(Overlay)
    overlay = image_rgb.copy()
    overlay[pred_mask > 0] = [255, 0, 0] # 마모 부위를 빨간색으로
    result = cv2.addWeighted(image_rgb, 0.7, overlay, 0.3, 0)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1); plt.imshow(image_rgb); plt.title("Original")
    plt.subplot(1, 3, 2); plt.imshow(pred_mask, cmap='gray'); plt.title("Predicted Mask")
    plt.subplot(1, 3, 3); plt.imshow(result); plt.title("Overlay Result")
    plt.show()

In [7]:
# 실행
if __name__ == "__main__":
    train()
    predict_and_show()

학습 시작 장치: cpu


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
